# Giving tools to LLM

In [1]:
#. import modules 
import os 
import gradio as gr 
from dotenv import load_dotenv 
from openai import OpenAI

In [2]:
# load environment variables 
load_dotenv(override=True)

True

In [3]:
# get the keys 
api_key=os.getenv("OPENAI_API_KEY")
gemini_api_key=os.getenv("GOOGLE_API_KEY")
ollama_api_key=os.getenv("OLLAMA_API_KEY")

# get the base url for gemini and ollama 
ollama_base_url=os.getenv("OLLAMA_BASE_URL")
gemini_base_url=os.getenv("GEMINI_BASE_URL")

In [4]:
# create model instances 
openai=OpenAI()
gemini=OpenAI(base_url=gemini_base_url, api_key=gemini_api_key)
ollama=OpenAI(base_url=ollama_base_url, api_key=ollama_api_key)

# Airline assistant

In [5]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence. If user has enquired for a list, give top 5 records only in a nice tabluar format. 
Always show Airline# or Airline ID, Airline name, Departure time, Arrival Time, Number of stops and Price for user to make selection for booking.
Always be accurate. If you don't know the answer, say so.
"""

In [6]:
# build the callback function for ai assistant 
def chat(message, history, model_name="mistral", model:OpenAI=ollama):
    history=[{"role": h["role"], "content": h["content"]} for h in history]

    # build the message with system message + history + user prompt 
    messages=[
        {"role": "system", "content": system_message} 
    ] + history + [
        {"role": "user", "content": message}
    ]

    response=model.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

### Give tools to AI assistant

In [7]:
import csv 

flight_data=[] 
flight_data_lower=[] 

with open("flight_dataset.csv", "r") as fp:
    reader=csv.DictReader(fp)
    for row in reader: 
        flight_data.append(row)

# convert all data to lower case 
for i in flight_data: 
    j= {} 
    for k, v in i.items():
        j[k.lower()]=v.lower()
    flight_data_lower.append(j)

flight_data_lower[0:2]



[{'airline_id': 'indigo-2',
  'airline': 'indigo',
  'source': 'bangalore',
  'destination': 'new delhi',
  'total_stops': '0',
  'price': '3897',
  'dep_hours': '22',
  'dep_min': '20',
  'arrival_hours': '1',
  'arrival_min': '10',
  'duration_hours': '2',
  'duration_min': '50'},
 {'airline_id': 'indigo-6',
  'airline': 'indigo',
  'source': 'bangalore',
  'destination': 'new delhi',
  'total_stops': '1',
  'price': '13302',
  'dep_hours': '16',
  'dep_min': '50',
  'arrival_hours': '21',
  'arrival_min': '35',
  'duration_hours': '4',
  'duration_min': '45'}]

In [8]:
from pydantic import BaseModel, Field

In [9]:
class BookTicketsInput(BaseModel):
    """ Make booking to particular airline """
    airline_number:str=Field(description="Airline unique number to make booking")

In [10]:
def book_ticket(airline_number):
    """ Function to make booking of the airline. Needs airline number as mandatory """

    for i in flight_data_lower:
        # print (i["airline"])
        if i["airline_id"]==airline_number.lower():
            return f'{i["airline_id"]} is booked for you'
        
    return "Unknown flight booking attempted"

In [11]:
class GetTicketPrice(BaseModel):
    """ Get list of ticket prices available for given source, destination and airline name """
    source:str=Field(description="Source city name from where travel will begin")
    destination:str=Field(description="Destination city where user intends to travel")
    airline:str=Field(description="Name of airline preferred by the user e.g. Indigo, Jet Airways etc...")

In [12]:
def get_ticket_price(source="banglore", destination="new delhi", airline=""): 
    """ Get list of avaiable airlines between source and desitination for given airline name """
    # print(f"Tool called to get ticket price for {source} to {destination} for {airline}")
    airline_list=[] 

    for i in flight_data_lower:
        # print (i["airline"])
        if i["airline"]==airline.lower() and i["source"]==source.lower() and i["destination"]==destination.lower():
            airline_lookup={
                "airline_id": i["airline_id"],
                "airline" : i["airline"], 
                "departure_time" : f"{i['dep_hours']}:{i['dep_min']} hours",
                "arrival_time": f"{i['arrival_hours']}:{i['arrival_min']} hours",
                "total_stops": i["total_stops"],
                "total_duration": f"{i['duration_hours']} hour(s) & {i['duration_min']} minutes",
                "price": i["price"]
            }
            airline_list.append(airline_lookup)
            # return f"The price of ticket from {source} to {destination} on {airline} is {i['price']}"
    return airline_list[:5]

In [13]:
get_ticket_price(source="bangalore", airline="indigo")

[{'airline_id': 'indigo-2',
  'airline': 'indigo',
  'departure_time': '22:20 hours',
  'arrival_time': '1:10 hours',
  'total_stops': '0',
  'total_duration': '2 hour(s) & 50 minutes',
  'price': '3897'},
 {'airline_id': 'indigo-6',
  'airline': 'indigo',
  'departure_time': '16:50 hours',
  'arrival_time': '21:35 hours',
  'total_stops': '1',
  'total_duration': '4 hour(s) & 45 minutes',
  'price': '13302'},
 {'airline_id': 'indigo-159',
  'airline': 'indigo',
  'departure_time': '18:55 hours',
  'arrival_time': '21:45 hours',
  'total_stops': '0',
  'total_duration': '2 hour(s) & 50 minutes',
  'price': '8855'},
 {'airline_id': 'indigo-163',
  'airline': 'indigo',
  'departure_time': '0:40 hours',
  'arrival_time': '3:25 hours',
  'total_stops': '0',
  'total_duration': '2 hour(s) & 45 minutes',
  'price': '4777'},
 {'airline_id': 'indigo-192',
  'airline': 'indigo',
  'departure_time': '12:10 hours',
  'arrival_time': '17:5 hours',
  'total_stops': '1',
  'total_duration': '4 hour(

In [14]:
class GetAvailableAirlines(BaseModel):
    """ Get list of all airlines and ticket prices available for given source and destination """
    source:str=Field(description="Source city name from where travel will begin")
    destination:str=Field(description="Destination city where user intends to travel")    

In [15]:
def get_available_airlines(source="bangalore", destination="new delhi"):
    """ get list of available airlines between source and destination for all airlines"""

    airline_list=[] 

    for i in flight_data_lower:
        if i["source"]==source.lower() and i["destination"]==destination.lower():
            airline_lookup={
                "airline_id": i["airline_id"],
                "airline" : i["airline"], 
                "departure_time" : f"{i['dep_hours']}:{i['dep_min']} hours",
                "arrival_time": f"{i['arrival_hours']}:{i['arrival_min']} hours",
                "total_stops": i["total_stops"],
                "total_duration": f"{i['duration_hours']} hour(s) & {i['duration_min']} minutes",
                "price": i["price"]
            }
            airline_list.append(airline_lookup)
    
    print(airline_list[:5])

    return airline_list[:5]

In [16]:
book_ticket

<function __main__.book_ticket(airline_number)>

In [17]:
get_ticket_price

<function __main__.get_ticket_price(source='banglore', destination='new delhi', airline='')>

In [18]:
get_available_airlines

<function __main__.get_available_airlines(source='bangalore', destination='new delhi')>

In [19]:
available_tools={
    "book_ticket": {"function" : book_ticket, "input_schema" : BookTicketsInput},
    "get_ticket_price": {"function" : get_ticket_price, "input_schema" : GetTicketPrice},
    "get_available_airlines" :{"function" : get_available_airlines, "input_schema" : GetAvailableAirlines},
}

In [20]:
tools = [
    {
        "type": "function",
        "function": {
            "name": name,
            "description": tool_info["input_schema"].__doc__,
            "parameters": tool_info["input_schema"].model_json_schema()
        }
    }
    for name, tool_info in available_tools.items()
]

In [21]:
tools

[{'type': 'function',
  'function': {'name': 'book_ticket',
   'description': ' Make booking to particular airline ',
   'parameters': {'description': 'Make booking to particular airline ',
    'properties': {'airline_number': {'description': 'Airline unique number to make booking',
      'title': 'Airline Number',
      'type': 'string'}},
    'required': ['airline_number'],
    'title': 'BookTicketsInput',
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': ' Get list of ticket prices available for given source, destination and airline name ',
   'parameters': {'description': 'Get list of ticket prices available for given source, destination and airline name ',
    'properties': {'source': {'description': 'Source city name from where travel will begin',
      'title': 'Source',
      'type': 'string'},
     'destination': {'description': 'Destination city where user intends to travel',
      'title': 'Destination',
      'type':

# create a callback function 

In [22]:
import json 

def handle_tool_calls(message):
    print ("tool called")
    response = [] 

    for tool_call in message.tool_calls:
        print (tool_call.function.name)
        tool_name=tool_call.function.name                                   # this will return function name in string format 
        arguments=json.loads(tool_call.function.arguments)                  # arguments returned by the LLM 
        tool = globals().get(tool_name)                                     # get the function from the globals varibale 
        result=tool(**arguments) if tool else {}
        response.append({
            "role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id
        })

    return response

# Models do not support tools 
1. gemma3 
2. deepseek-r1
3. mistral - tool calling supported but not even calling tool
4. phi4-mini - not even calling tool 
5. llama3.1 and 3.2 - both sometimes work and something they dont
6. phi3:medium

In [37]:
# def chat(message, history, model_name="gpt-4.1-mini", model=openai):
def chat(message, history, model_name="llama3.1", model=ollama):    
    history=[{"role": h["role"], "content": h["content"]} for h in history]
    print (history)

    # build the message with system message + history + user prompt 
    messages=[
        {"role": "system", "content": system_message} 
    ] + history + [
        {"role": "user", "content": message}
    ]

    # call the model , passing the tool in chat completion 
    response=model.chat.completions.create(model=model_name, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = model.chat.completions.create(model=model_name, messages=messages, tools=tools)

    return response.choices[0].message.content

In [38]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


[]
tool called
get_available_airlines
[{'airline_id': 'indigo-2', 'airline': 'indigo', 'departure_time': '22:20 hours', 'arrival_time': '1:10 hours', 'total_stops': '0', 'total_duration': '2 hour(s) & 50 minutes', 'price': '3897'}, {'airline_id': 'indigo-6', 'airline': 'indigo', 'departure_time': '16:50 hours', 'arrival_time': '21:35 hours', 'total_stops': '1', 'total_duration': '4 hour(s) & 45 minutes', 'price': '13302'}, {'airline_id': 'jet airways-8', 'airline': 'jet airways', 'departure_time': '18:55 hours', 'arrival_time': '10:25 hours', 'total_stops': '1', 'total_duration': '15 hour(s) & 30 minutes', 'price': '11087'}, {'airline_id': 'jet airways-9', 'airline': 'jet airways', 'departure_time': '8:0 hours', 'arrival_time': '5:5 hours', 'total_stops': '1', 'total_duration': '21 hour(s) & 5 minutes', 'price': '22270'}, {'airline_id': 'jet airways-10', 'airline': 'jet airways', 'departure_time': '8:55 hours', 'arrival_time': '10:25 hours', 'total_stops': '1', 'total_duration': '25 ho

# Database Option

In [39]:
import sqlite3

In [40]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS flight_dataset (
	"Airline_Id"	TEXT,
	"Airline"	TEXT,
	"Source"	TEXT,
	"Destination"	TEXT,
	"Total_Stops"	INTEGER,
	"Price"	INTEGER,
	"Dep_hours"	INTEGER,
	"Dep_min"	INTEGER,
	"Arrival_hours"	INTEGER,
	"Arrival_min"	INTEGER,
	"Duration_hours"	INTEGER,
	"Duration_min"	INTEGER
    )''')
    conn.commit()

In [41]:
def get_ticket_price(source="bangalore", destination="new delhi", airline=""): 
    """ Get list of avaiable airlines between source and desitination for given airline name """
    # print(f"Tool called to get ticket price for {source} to {destination} for {airline}")

    returnable=[]

    with sqlite3.connect(DB) as conn:
        cursor=conn.cursor()
        cursor.execute("""SELECT Airline_Id, Airline, (dep_hours || ':' || dep_min) as Departure_Time, (arrival_hours || ':' || arrival_min) as arrival_time, total_stops, (duration_hours || ' hours & ' || duration_min || ' minutes') as total_duration, price
                            FROM flight_dataset 
                           WHERE lower(Airline)=? and lower(source)=? and lower(destination)=?
                       order by total_stops, price, departure_time, arrival_time
                       """, (airline.lower(), source.lower(), destination.lower())) 
        
        # fetch 10 records 
        rows = cursor.fetchmany(10)

        if rows: 
            for row in rows:
                returnable.append(
                    {
                        "airline_id": row[0],
                        "airline" : row[1],
                        "departure_time" : row[2],
                        "arrival_time": row[3],
                        "total_stops": row[4],
                        "total_duration": row[5],
                        "price": row[6]
                    }
                )

        return returnable
        

In [42]:
get_ticket_price(source="bangalore", airline="indigo")

[{'airline_id': 'IndiGo-2',
  'airline': 'IndiGo',
  'departure_time': '22:20',
  'arrival_time': '1:10',
  'total_stops': 0,
  'total_duration': '2 hours & 50 minutes',
  'price': 3897},
 {'airline_id': 'IndiGo-5952',
  'airline': 'IndiGo',
  'departure_time': '0:40',
  'arrival_time': '3:25',
  'total_stops': 0,
  'total_duration': '2 hours & 45 minutes',
  'price': 4238},
 {'airline_id': 'IndiGo-7498',
  'airline': 'IndiGo',
  'departure_time': '1:30',
  'arrival_time': '4:15',
  'total_stops': 0,
  'total_duration': '2 hours & 45 minutes',
  'price': 4238},
 {'airline_id': 'IndiGo-54',
  'airline': 'IndiGo',
  'departure_time': '22:20',
  'arrival_time': '1:10',
  'total_stops': 0,
  'total_duration': '2 hours & 50 minutes',
  'price': 4377},
 {'airline_id': 'IndiGo-2459',
  'airline': 'IndiGo',
  'departure_time': '22:20',
  'arrival_time': '1:10',
  'total_stops': 0,
  'total_duration': '2 hours & 50 minutes',
  'price': 4377},
 {'airline_id': 'IndiGo-9197',
  'airline': 'IndiGo'

In [43]:
def get_available_airlines(source="bangalore", destination="new delhi"): 
    """ get list of available airlines between source and destination for all airlines"""
    # print(f"Tool called to get ticket price for {source} to {destination} for {airline}")
    returnable=[]
    with sqlite3.connect(DB) as conn:
        cursor=conn.cursor()
        cursor.execute("""SELECT Airline_Id, Airline, (dep_hours || ':' || dep_min) as Departure_Time, (arrival_hours || ':' || arrival_min) as arrival_time, total_stops, (duration_hours || ' hours & ' || duration_min || ' minutes') as total_duration, price
                            FROM flight_dataset 
                           WHERE lower(source)=? and lower(destination)=?
                       order by total_stops, price, departure_time, arrival_time
                       """, (source.lower(), destination.lower())) 
        
        rows = cursor.fetchmany(10)
        if rows: 
            for row in rows:
                returnable.append(
                    {
                        "airline_id": row[0],
                        "airline" : row[1],
                        "departure_time" : row[2],
                        "arrival_time": row[3],
                        "total_stops": row[4],
                        "total_duration": row[5],
                        "price": row[6]
                    }
                )


        return returnable

In [44]:
get_available_airlines(source="bangalore", destination="new delhi")

[{'airline_id': 'Air Asia-9786',
  'airline': 'Air Asia',
  'departure_time': '11:10',
  'arrival_time': '13:55',
  'total_stops': 0,
  'total_duration': '2 hours & 45 minutes',
  'price': 3383},
 {'airline_id': 'Air Asia-10592',
  'airline': 'Air Asia',
  'departure_time': '23:25',
  'arrival_time': '2:10',
  'total_stops': 0,
  'total_duration': '2 hours & 45 minutes',
  'price': 3383},
 {'airline_id': 'Air Asia-1119',
  'airline': 'Air Asia',
  'departure_time': '5:50',
  'arrival_time': '8:40',
  'total_stops': 0,
  'total_duration': '2 hours & 50 minutes',
  'price': 3383},
 {'airline_id': 'SpiceJet-47',
  'airline': 'SpiceJet',
  'departure_time': '5:45',
  'arrival_time': '8:35',
  'total_stops': 0,
  'total_duration': '2 hours & 50 minutes',
  'price': 3527},
 {'airline_id': 'SpiceJet-5484',
  'airline': 'SpiceJet',
  'departure_time': '5:45',
  'arrival_time': '8:35',
  'total_stops': 0,
  'total_duration': '2 hours & 50 minutes',
  'price': 3579},
 {'airline_id': 'Air India-1

In [45]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


[]
tool called
get_available_airlines
